# Demonstration of WellResponse Class

Notebook to demonstrate using the Well Class to 
estimate drawdown or stream depletion using the 
solutions available in the python module.

In [1]:
import sys
sys.path.insert(1, '../../')
import pycap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Example Depletion
The Well class will estimate depletions for streams.  The stream distances and
streambed conductances, if needed, are passed through dictionaries keyed by 
a stream name or ID.  In this notebook a table of values will be made manually from 
Table 2 from Reeves, H.W., Hamilton, D.A., Seelbach, P.W., and Asher, A.J., 2009, Ground-water-withdrawal component of the Michigan water-withdrawal screening tool: U.S. Geological Survey Scientific Investigations Report 2009–5003, 36 p.
[https://pubs.usgs.gov/sir/2009/5003/]

In [2]:
stream_table = pd.DataFrame(({'id': 8, 'distance': 14802},
                            {'id': 9, 'distance': 12609.2},
                            {'id': 11, 'distance': 15750.5},
                            {'id': 27, 'distance': 22567.6},
                            {'id': 9741, 'distance': 27565.2},
                            {'id': 10532, 'distance': 33059.5},
                            {'id': 11967, 'distance': 14846.3},
                            {'id': 12515, 'distance': 17042.5},
                            {'id': 12573, 'distance': 11959.5},
                            {'id': 12941, 'distance': 19070.8},
                            {'id': 13925, 'distance': 10028.9}))



In [3]:
stream_table.head()

,id,distance
0,8,14802.0
1,9,12609.2
2,11,15750.5
3,27,22567.6
4,9741,27565.2


# Compute apportionment

The Michigan screening tool uses inverse distance weighting apportionment.  Other
apportionment approaches that may be used as a simple way to extend the analytical
solution are discussed by Zipper and others (2019).  [https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2018WR024403]

In [4]:
invers =np.array([1/x for x in stream_table['distance']])
stream_table['apportionment'] = (1./stream_table['distance'])/np.sum(invers)

In [5]:
stream_table

,id,distance,apportionment
0,8,14802.00,0.098916
1,9,12609.20,0.116118
2,11,15750.50,0.092959
3,27,22567.60,0.064878
4,9741,27565.20,0.053116
5,10532,33059.50,0.044288
6,11967,14846.30,0.098621
7,12515,17042.55,0.085911
8,12573,11959.50,0.122426
9,12941,19070.80,0.076774


In [6]:
np.sum(stream_table['apportionment'])

np.float64(1.0000000000000002)

make dictionaries from the distances and apportionment values

In [7]:
distances = dict(zip(stream_table.id.values, stream_table.distance.values))

In [8]:
apportion = dict(zip(stream_table.id.values, stream_table.apportionment.values))

# set aquifer properties and streambed conductances

In [9]:
stream_table = pd.DataFrame(({'id': 8, 'distance': 14802},
                            {'id': 9, 'distance': 12609.2},
                            {'id': 11, 'distance': 15750.5},
                            {'id': 27, 'distance': 22567.6},
                            {'id': 9741, 'distance': 27565.2},
                            {'id': 10532, 'distance': 33059.5},
                            {'id': 11967, 'distance': 14846.3},
                            {'id': 12515, 'distance': 17042.5},
                            {'id': 12573, 'distance': 11959.5},
                            {'id': 12941, 'distance': 19070.8},
                            {'id': 13925, 'distance': 10028.9}))

In [10]:
T= 7211.  # ft^2/day
S= 0.01
Q = 70  # 70 gpm in cubic feet per day
well_name = 'demo'
pumpdays = int(5. * 365)

## call a utility to convert Q to a series with appropriate formatting

the utility also converts to CFD from GPM

In [11]:
Q = pycap.Q2ts(pumpdays, 5, Q)

In [12]:
Q

1       13475.935829
2       13475.935829
3       13475.935829
4       13475.935829
5       13475.935829
            ...     
1821    13475.935829
1822    13475.935829
1823    13475.935829
1824    13475.935829
1825    13475.935829
Length: 1825, dtype: float64

In [13]:
stream_table['conductance'] = 7.11855  # example uses constant streambed_conductance

In [14]:
stream_table.head()

,id,distance,conductance
0,8,14802.0,7.11855
1,9,12609.2,7.11855
2,11,15750.5,7.11855
3,27,22567.6,7.11855
4,9741,27565.2,7.11855


In [15]:
cond = dict(zip(stream_table.id.values, stream_table.conductance.values))

In [16]:
test_well = pycap.Well(T=T,
                 S=S,
                 Q=Q,
                 depletion_years=5,
                 depl_method='hunt99',
                 streambed_conductance=cond,
                 stream_dist=distances,
                 stream_apportionment=apportion)

In [17]:
deltaQ = pycap._calc_deltaQ(Q)
deltaQ

1    13475.935829
dtype: float64

In [18]:
test_well.depletion

c:\Home-Local\pycap\source\examples\../..\pycap\solutions.py:558: RuntimeWarning: divide by zero encountered in divide
  a = np.sqrt(S * dist**2 / (4.0 * T * time))


{np.int64(8): array([0.00000000e+00, 4.61384782e-38, 3.83338737e-21, ...,
        1.14615834e-02, 1.14626313e-02, 1.14636783e-02], shape=(1825,)),
 np.int64(9): array([0.00000000e+00, 8.25656693e-29, 2.02230810e-16, ...,
        1.40440789e-02, 1.40451628e-02, 1.40462458e-02], shape=(1825,)),
 np.int64(11): array([0.00000000e+00, 1.66711353e-42, 2.11329070e-23, ...,
        1.05692983e-02, 1.05703321e-02, 1.05713651e-02], shape=(1825,)),
 np.int64(27): array([0.00000000e+00, 2.70335278e-82, 1.61669806e-43, ...,
        6.39194779e-03, 6.39289278e-03, 6.39383705e-03], shape=(1825,)),
 np.int64(9741): array([0.00000000e+000, 2.82691069e-120, 1.23911560e-062, ...,
        4.67310017e-003, 4.67398441e-003, 4.67486800e-003], shape=(1825,)),
 np.int64(10532): array([0.00000000e+000, 1.16079177e-170, 6.09674630e-088, ...,
        3.41265831e-003, 3.41347546e-003, 3.41429203e-003], shape=(1825,)),
 np.int64(11967): array([0.00000000e+00, 2.90079979e-38, 3.02696360e-21, ...,
        1.14173413e

In [19]:
stream_depl = pd.DataFrame(test_well.depletion)

## need to convert to GPM to compare results to Table 2

In [20]:
stream_depl = stream_depl * 448.83116885

In [21]:
stream_depl

,8,9,11,27,9741,10532,11967,12515,12573,12941,13925
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,2.070839e-35,3.705805e-26,7.482525e-40,1.213349e-79,1.268806e-117,5.209995e-168,1.301969e-35,2.478604e-46,1.093430e-23,1.668067e-57,4.471805e-17
2,1.720544e-18,9.076749e-14,9.485107e-21,7.256245e-41,5.561537e-60,2.736410e-85,1.358596e-18,4.887885e-24,1.675473e-12,1.081763e-29,4.289447e-09
3,9.397878e-13,1.521391e-09,2.774879e-14,7.755561e-28,1.159949e-40,1.310414e-57,8.007453e-13,1.661491e-16,1.112159e-08,2.547248e-20,2.414348e-06
4,7.690689e-10,2.171971e-07,5.263075e-11,2.831976e-21,5.935178e-31,1.018381e-43,6.807403e-10,1.076097e-12,9.977078e-07,1.376326e-15,6.274232e-05
...,...,...,...,...,...,...,...,...,...,...,...
1820,5.143374e+00,6.302446e+00,4.742901e+00,2.868056e+00,2.096638e+00,1.530973e+00,5.123517e+00,4.269962e+00,6.728167e+00,3.658731e+00,8.320658e+00
1821,5.143845e+00,6.302933e+00,4.743366e+00,2.868481e+00,2.097036e+00,1.531340e+00,5.123988e+00,4.270419e+00,6.728660e+00,3.659175e+00,8.321169e+00
1822,5.144316e+00,6.303420e+00,4.743830e+00,2.868905e+00,2.097433e+00,1.531707e+00,5.124459e+00,4.270875e+00,6.729152e+00,3.659619e+00,8.321680e+00
1823,5.144786e+00,6.303907e+00,4.744294e+00,2.869330e+00,2.097830e+00,1.532074e+00,5.124929e+00,4.271331e+00,6.729644e+00,3.660063e+00,8.322190e+00


In [26]:
five_year= pd.DataFrame(stream_depl.loc[1824].T)

In [29]:
five_year.rename(columns={1824: 'Depletion'}, inplace=True)

In [31]:
pd.options.display.float_format = '{:,.2f}'.format
five_year

,Depletion
8,5.15
9,6.30
11,4.74
27,2.87
9741,2.10
10532,1.53
11967,5.13
12515,4.27
12573,6.73
12941,3.66


# Matches Table 2